In [ ]:
import re
with open('scop_entire.txt','r') as f:
    f1 = f.readlines()

def extract_class(ClassCode,Name):
    g = open(f"{Name}.txt",'w')
    for i in f1:
        se = re.search(f"^8\d+\s\d...\s.*\,CL={ClassCode}\,CF.*",i)
        if se:
            g.write(i)
            
extract_class(1000000,'all_alpha')
extract_class(1000001,'all_beta')
extract_class(1000002,'alpha_beta')
extract_class(1000003,'alpha_plus_beta')

#extracting PDB IDs class wise
def extract_pdb(file_name,list_name):
    with open(f"{file_name}.txt",'r') as fl:
        f2 = fl.readlines()
    for i in f2:
        se2 = re.search('^8\d+\s(\d...)\s\w\:.*',i)
        if se2:
            list_name.append(str(se2.group(1)))
alpha, beta, alp_bta, alp_p_bta = [], [], [], []            
extract_pdb('all_alpha',alpha)
extract_pdb('all_beta',beta)
extract_pdb('alpha_beta',alp_bta)
extract_pdb('alpha_plus_beta',alp_p_bta)


In [ ]:
#data extraction and filtering
import urllib.request
import os
import math

# folder path
download_foldera = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\All_Alpha_Scop\\Complete"
download_folderb = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\All_Alpha_Scop\\Chain_break"
download_folderc = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\All_Beta_Scop\\Complete"
download_folderd = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\All_Beta_Scop\\Chain_break"
#download_foldere = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\AlphaBeta_Scop\\Complete"
#download_folderf = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\AlphaBeta_Scop\\Chain_break"
#download_folderg = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\Alpha_pBeta_Scop\\Complete"
#download_folderh = "C:\\Users\\chaud\\Music\\Project\\PISCES\\2.5R\\Alpha_pBeta_Scop\\Chain_break"

# Create the folder 
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        
create_folder(download_foldera)
create_folder(download_folderb)
create_folder(download_folderc)
create_folder(download_folderd)
#create_folder(download_foldere)
#create_folder(download_folderf)
#create_folder(download_folderg)
#create_folder(download_folderh)

x, y, z, atom, complete, cb = [], [], [], [], [], []
import urllib.request
def extract_file(pdb_list,folder_path,folder_path_cb):
    a = 0
    b=0
    c=0
    cc=0
    for i in pdb_list:
        with urllib.request.urlopen(f"https://files.rcsb.org/view/{i}.cif") as response:
            content = response.read().decode('utf-8')
            content = content.splitlines()
            for k in content:
                x1 = re.search(r"^ATOM\s+\d+\s+\w\s+(N|C)\s+[AD-Z\.]\s+\w+.*\?\s+(.*\d+\.\d+)\s+(.*\d+\.\d+)\s+(.*\d+\.\d+)\s+[0-1]\..*\s+\d+.*\?.*",k)
                if x1:
                    atom.append(x1.group(1))
                    x.append(float(x1.group(2)))
                    y.append(float(x1.group(3)))
                    z.append(float(x1.group(4)))
            if len(x)>1:   
                content.clear()
                x.pop(0)
                y.pop(0)
                z.pop(0)
                for j in range(0,(len(x)-1),2):
                    p = [x[j], y[j], z[j]]
                    q = [x[j+1], y[j+1], z[j+1]]
                    d = (math.dist(p,q))
                    #check for chain breaks (data filtering)
                    if d>1.42:
                        a+=1
                    elif atom[j]==atom[j+1] or atom[j+1]==atom[j]:
                        b+=1
                x.clear()
                y.clear()
                z.clear()
                atom.clear()
                if a == 0 and b == 0:
                    print(i)
                    file_url = f"https://files.rcsb.org/view/{i}.cif"
                    file_path = os.path.join(folder_path, f"{i}.cif")
                    urllib.request.urlretrieve(file_url, file_path)
                    complete.append(i)
                    a=0
                    b=0
                else:
                    print(i,"cb")
                    file_urll = f"https://files.rcsb.org/view/{i}.cif"
                    file_pathh = os.path.join(folder_path_cb, f"{i}.cif")
                    urllib.request.urlretrieve(file_urll, file_pathh)
                    cb.append(i)
                    a=0
                    b=0

    with open(f"{folder_path}\\Complete.txt",'w') as w1:
        for i in complete:
            w1.write(f"{i}\n")
    w1.close()

    with open(f"{folder_path_cb}\\ChainBreak.txt",'w') as w2:
        for i in cb:
            w2.write(f"{i}\n")
    w2.close()

extract_file(alpha, download_foldera, download_folderb)
extract_file(beta, download_folderc, download_folderd)
#extract_file(alp_bta, download_foldere, download_folderf)
#extract_file(alp_p_bta, download_folderg, download_folderh)


    